## Linear Regression using a practical Example

We will use a insurance [dataset](https://www.kaggle.com/mirichoi0218/insurance) from kaggle.
This will be not provided on ISIS. If you want to run the code on your own, please download it directly from kaggle.

Note that you do not need to understand the code, even if it is mostly straight foreward. This is more about visualizing a data set, think about it and explore how we could/can apply our methods.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

In [ ]:
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('insurance.csv')
data.head()

In [ ]:
data.describe()

What could we do with this data set?

Different types of classes:
* binary classes: sex, smoker
* classes: sex, region
* floating point features: bmi, charges
* integer features: childs, age (at least represented as an integer)
* ...

In [ ]:
A = data.loc[data.smoker == 'yes', ['bmi', 'charges']]
B = data.loc[data.smoker == 'no', ['bmi', 'charges']]

plt.figure(figsize=(10,) * 2)
plt.scatter(A.bmi, A.charges, s=25, alpha=.7, label='smoker')
plt.scatter(B.bmi, B.charges, s=25, alpha=.7, label='non-smoker')
plt.legend()
plt.xlabel('BMI')
plt.ylabel('charges')
plt.show()

* We could train a linear classifier on the data
\begin{align}
    & f: \mathbb{R}^d \to \mathcal{C} \\
    & f: \mathbb{R}^d \to \{\text{smoker}, \text{non-smoker}\} \\
    & f(\mathcal{x}) = \mathbf{w}^\top\mathbf{x}-\beta \geq 0
\end{align}

* Which one might be suitable?
    * NCC
    * LDA
    * Perceptron
* Data is scattered, not gaussian and classes have no common covariance<br><br>
 
* For smokers: linear dependency between BMI and charges<br><br>

* What else could have a influence on the charge?

In [ ]:
data.head()

In [ ]:
A = data.loc[data.smoker == 'yes', ['age', 'charges']]
B = data.loc[data.smoker == 'no', ['age', 'charges']]

plt.figure(figsize=(10,) * 2)
plt.scatter(A.age, A.charges, s=25, alpha=.7, label='smoker')
plt.scatter(B.age, B.charges, s=25, alpha=.7, label='non-smoker')
plt.legend()
plt.xlabel('age')
plt.ylabel('charges')
plt.show()

* Again, we could train a binary classifier<br><br>

* **strong** linear dependency between age and charges<br><br>

* how do we measure that? covariance (the magnitude does not tell a lot!)

In [ ]:
Sx = np.cov(data.loc[:, ['age', 'charges']].T)
print("Covariance")
print(Sx)
print("Correlation between age and charges:",
      Sx[0, 1] / np.prod(np.sqrt(np.diag(Sx))))

#### Can we predict the dependency between age and charges?

Linear Regression
\begin{align}
    & f: \mathbb{R}^d \to \mathbb{R} \\
    & f(\mathcal{x}) = \mathbf{w}^\top\mathbf{x}-\beta
\end{align}

In [ ]:
A = data.loc[data.smoker == 'yes', ['age', 'charges']]
B = data.loc[data.smoker == 'no', ['age', 'charges']]


regA = LinearRegression().fit(np.expand_dims(A.age, axis=1), A.charges)
regB = LinearRegression().fit(np.expand_dims(B.age, axis=1), B.charges)

x = np.linspace(A.age.min(), A.age.max(), 200)
yA = regA.predict(np.expand_dims(x, axis=1))
yB = regB.predict(np.expand_dims(x, axis=1))


plt.figure(figsize=(10,) * 2)

plt.plot(x, yA)
plt.scatter(A.age, A.charges, s=25, alpha=.7, label='smoker')

plt.plot(x, yB)
plt.scatter(B.age, B.charges, s=25, alpha=.7, label='non-smoker')

plt.legend()
plt.xlabel('age')
plt.ylabel('charges')
plt.show()

Does it make sense to predict the charge given the age by linear regression?

* age is binned in integer values
* when we want to approximate the average charges given an age, we could just take the average for the age.<br><br>

* If we want to approximation e.g. the age $25.5$, regression is probably useful: it describes the linear dependency between age and charges

In [ ]:
A = data.loc[data.smoker == 'yes', ['bmi', 'charges']]
B = data.loc[data.smoker == 'no', ['bmi', 'charges']]


regA = LinearRegression().fit(np.expand_dims(A.bmi, axis=1), A.charges)
regB = LinearRegression().fit(np.expand_dims(B.bmi, axis=1), B.charges)

x = np.linspace(A.bmi.min(), A.bmi.max(), 200)
yA = regA.predict(np.expand_dims(x, axis=1))
yB = regB.predict(np.expand_dims(x, axis=1))


plt.figure(figsize=(10,) * 2)

plt.plot(x, yA)
plt.scatter(A.bmi, A.charges, s=25, alpha=.7, label='smoker')

plt.plot(x, yB)
plt.scatter(B.bmi, B.charges, s=25, alpha=.7, label='non-smoker')

plt.legend()
plt.xlabel('BMI')
plt.ylabel('charges')
plt.show()

* here regression could be more useful
* if we want to get approximate charges, we can use the regressed function<br><br>

* taking a look at the non-smokers, we can see that there is also a small linear dependency
* how do we check that? again, covariance!

In [ ]:
Sx = np.cov(data.loc[data.smoker == 'no', ['bmi', 'charges']].T)
print("Covariance")
print(Sx)
print("Correlation between age and charges:",
      Sx[0, 1] / np.prod(np.sqrt(np.diag(Sx))))

## Task 2 - Variance of OLS Estimation

Here I creted the algorithm of the task sheet for some empirical experiments.

In [2]:
def Ex3Algorithm(w, n, vd, vn, trials):
    # generate gaussian data samples
    x = np.random.normal(0, vd, n)
    # repeat them to add different noise values
    X = np.repeat(x[None], trials, axis=0)
    # generate noise for each sample in each trial
    E = np.random.normal(0, vn, (trials, n))
    # calculate y values and add the noise
    Y = w * X + E
    # calculate the approximative slope for each trial
    Wapprox = np.sum(X * Y, axis=1) / np.sum(X * X, axis=1)
    # return the variance of the approximative slopes
    return np.var(Wapprox)

def Ex3AlgorithmTester(w, n, vd, vn, trials=10**5, iterations=100):
    """
    This runs the algorithm described in exercise sheet 03 task 2
    Input:
        w             true slope to calculate y = w * x
        n             number of samples x
        vd            variance used to drap the samples x
        vn            variance of the noise added to y
        trials        iterations of the algorithm (19**5 on the task sheet)
        iterations    how often the algorithm is launched
    Output:
        mean of the variances returned by the algorithm
    """
    variances = []
    for i in range(iterations):
        variances.append(Ex3Algorithm(w, n, vd, vn, trials))
        if i % 10 == 0:
            print("\r({}/{})".format(i, iterations), end='', flush=True)
    
    print("\r", end='', flush=True)
    return np.mean(variances)

The following cells might need some time.

First, test for different values of the true slope $w$.

In [3]:
print("w=1000; n=100; var data=0.5; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(1000, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 100, 0.5, 1.0)))

w=1000; n=100; var data=0.5; var noise=1.0
0.042108
0.041271
0.040384
0.041174


In [8]:
print("w=1; n=100; var data=0.5; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(1, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1, 100, 0.5, 1.0)))

w=1; n=100; var data=0.5; var noise=1.0
0.042360
0.040862
0.040993
0.039833


In [4]:
print("w=0.001; n=100; var data=0.5; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 100, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 100, 0.5, 1.0)))

w=0.001; n=100; var data=0.5; var noise=1.0
0.041296
0.040272
0.041118
0.041794


There seem to be no significant difference.

---

Now alternate the number of samples $n$.

In [5]:
print("w=0.001; n=200; var data=0.5; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 0.5, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 0.5, 1.0)))

w=0.001; n=200; var data=0.5; var noise=1.0
0.020560
0.020212
0.020028
0.019886


There is a noticable difference!

---

Now alternate the data variance $\sigma^2_x$.

In [6]:
print("w=0.001; n=200; var data=2.0; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(0.001, 200, 2.0, 1.0)))

w=0.001; n=200; var data=2.0; var noise=1.0
0.001253
0.001278
0.001261
0.001274


There is a noticable difference!

---

Now alternate again the true slope $w$.

In [7]:
print("w=1000; n=200; var data=2.0; var noise=1.0")
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 1.0)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 1.0)))

w=1000; n=200; var data=2.0; var noise=1.0
0.001255
0.001255
0.001239
0.001264


There seem to be no significant difference.

---

Now alternate the noise variance $\sigma^2_\epsilon$.

In [9]:
print("w=1000; n=200; var data=2.0; var noise=0.5")
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 0.5)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 0.5)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 0.5)))
print("{:.6f}".format(Ex3AlgorithmTester(1000, 200, 2.0, 0.5)))

w=1000; n=200; var data=2.0; var noise=0.5
0.000317
0.000318
0.000308
0.000315


There is a noticable difference!